# FinBERT Profiling: LoRA

This notebook reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.

The purpose of this notebook is to profile training the original FinBERT model.



In [14]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data, timed_eval
import finbert.utils as tools


%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
wandb.init(
    entity="si2449-columbia-university",
    project="Project-runs",
    name="training-lora-profiled",
    group="lora"
)

In [16]:
cl_path = project_dir/'models'/'teacher_lora'
cl_data_path = project_dir/'data'/'sentiment_data'

In [17]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', cache_dir=None, num_labels=3)

config = Config(
    data_dir=cl_data_path,
    bert_model=bertmodel,

    learning_rate=0.0009640727001180892,
    max_seq_length=48,
    num_train_epochs=10,
    train_batch_size=32,
    warm_up_proportion=0.26534125891570404,

    use_lora=True,
    lora_r=8,
    lora_alpha=8,
    lora_dropout=0.02385391306485725,
    lora_target_modules=["query", "value", "dense"],

    model_dir=cl_path,
    output_mode="classification",
    local_rank=-1,

    # Keep these off for LoRA
    discriminate=False,
    gradual_unfreeze=False,
)

config.profile_train_steps = 20

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
finbert = ProfiledFinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [19]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/20/2025 01:45:11 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [20]:
train_data = finbert.get_data('train')

In [21]:
model = finbert.create_the_model()

trainable params: 1,194,243 || all params: 110,678,790 || trainable%: 1.0790


In [22]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start



12/20/2025 01:45:13 - INFO - finbert.utils -   *** Example ***
12/20/2025 01:45:13 - INFO - finbert.utils -   guid: train-1
12/20/2025 01:45:13 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/20/2025 01:45:13 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:45:13 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:45:13 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/

12/20/2025 01:45:14 - INFO - finbert.finbert -   ***** Loading data *****
12/20/2025 01:45:14 - INFO - finbert.finbert -     Num examples = 3488
12/20/2025 01:45:14 - INFO - finbert.finbert -     Batch size = 32
12/20/2025 01:45:14 - INFO - finbert.finbert -     Num steps = 120



Starting Profiled Training
Device: cuda
Profiling activities: [<ProfilerActivity.CPU: 0>, <ProfilerActivity.CUDA: 2>]



Epoch:   0%|          | 0/10 [00:04<?, ?it/s]



Profiling complete for first epoch (20 steps)
Continuing full training without profiling...


PROFILING RESULTS - Training


By CPU Time:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  cudaStreamSynchronize        35.92%        1.859s        35.92%

...

Validation losses: [0.5687525891340696, 0.36604770674155307, 0.3356134914434873, 0.3157747628597113, 0.3708775501984816, 0.48568138308249986, 0.44731278946766484, 0.5241895140363619]


Iteration: 100%|██████████| 109/109 [00:22<00:00,  4.82it/s]
12/20/2025 01:49:55 - INFO - finbert.utils -   *** Example ***
12/20/2025 01:49:55 - INFO - finbert.utils -   guid: validation-1
12/20/2025 01:49:55 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/20/2025 01:49:55 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:49:55 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:49:55 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:49:55 - INFO - finbert.utils -   label: neutral (id = 2)
12/20/2025 01:49:56 

Validation losses: [0.5687525891340696, 0.36604770674155307, 0.3356134914434873, 0.3157747628597113, 0.3708775501984816, 0.48568138308249986, 0.44731278946766484, 0.5241895140363619, 0.567912276261128]


Iteration: 100%|██████████| 109/109 [00:22<00:00,  4.82it/s]
12/20/2025 01:50:19 - INFO - finbert.utils -   *** Example ***
12/20/2025 01:50:19 - INFO - finbert.utils -   guid: validation-1
12/20/2025 01:50:19 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/20/2025 01:50:19 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:19 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:19 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:19 - INFO - finbert.utils -   label: neutral (id = 2)
12/20/2025 01:50:19 

Validation losses: [0.5687525891340696, 0.36604770674155307, 0.3356134914434873, 0.3157747628597113, 0.3708775501984816, 0.48568138308249986, 0.44731278946766484, 0.5241895140363619, 0.567912276261128, 0.5606953891424032]


In [23]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)


eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)


12/20/2025 01:50:22 - INFO - finbert.utils -   *** Example ***
12/20/2025 01:50:22 - INFO - finbert.utils -   guid: test-1
12/20/2025 01:50:22 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/20/2025 01:50:22 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:22 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:22 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:22 - IN

Testing: 100%|██████████| 31/31 [00:02<00:00, 10.75it/s]
12/20/2025 01:50:25 - INFO - finbert.utils -   *** Example ***
12/20/2025 01:50:25 - INFO - finbert.utils -   guid: test-1
12/20/2025 01:50:25 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/20/2025 01:50:25 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:25 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/20/2025 01:50:25 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [24]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Evaluation Loss:{0:.2f}".format(loss))
    print("Evaluation Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    print(return_val)
    return_val["Evalaution Loss"] = loss
    return_val["Evaluation Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [25]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [26]:
wandb_report = report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
    }

wandb.log(summary)
wandb.finish()


Evaluation Loss:0.45
Evaluation Accuracy:0.85

Classification Report:
{'0': {'precision': 0.7735191637630662, 'recall': 0.8314606741573034, 'f1-score': 0.8014440433212996, 'support': 267.0}, '1': {'precision': 0.7718120805369127, 'recall': 0.8984375, 'f1-score': 0.8303249097472925, 'support': 128.0}, '2': {'precision': 0.9101123595505618, 'recall': 0.8452173913043478, 'f1-score': 0.8764652840396754, 'support': 575.0}, 'accuracy': 0.8484536082474227, 'macro avg': {'precision': 0.8184812012835136, 'recall': 0.8583718551538837, 'f1-score': 0.8360780790360892, 'support': 970.0}, 'weighted avg': {'precision': 0.854264092551584, 'recall': 0.8484536082474227, 'f1-score': 0.8497264807600555, 'support': 970.0}}


Evalaution Loss,▁
Evaluation Accuracy,▁
accuracy,▁
epoch,▁▁▁▁▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇███
eval_num_samples,▁
eval_samples_per_s,▁
eval_wall_s,▁
learning_rate,▂▂▂▃▃▄▄▅▇▇███▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▃▃▃▃▂▂▁▁▁
model_size_mb,▁
profile_train_steps,▁
+11,...
